In [ ]:
import numpy as np
import lightcones as lc
import lightcones.linalg as la
import matplotlib.pyplot as plt
from lightcones.linalg import mv
from lightcones import models
from lightcones.jumps import make_jump
from lightcones.solvers.schrodinger import solve

In [ ]:
import random
from lightcones.jumps import density_matrix

def causal_diamond_frame_dynamic(spread_min, times_in, U_min, rho_plus_min, dt, rtol):
    spread_cd = np.copy(spread_min)
    rho_minus = np.copy(rho_plus_min)

    U_cd = []         
    times_out = [] 

    ntg = spread_cd.shape[1]
    n_out = 0

    for ti in range(ntg):
        n_in = lc.m_in(times_in, ti)

        if n_in - n_out > 0:
            la.make_hermitean(rho_minus)
            rho_cd = rho_minus[n_out:n_in, n_out:n_in]

            pi, U = la.find_eigs_ascending(rho_cd)
            pi_min, pi_max = np.real(pi[0]), np.real(pi[-1])
            g_metric = pi_min - rtol * pi_max

            if g_metric < 0:
                times_out.append(ti)

                spread_cd[n_out:n_in, ti:] = U.T.conj() @ spread_cd[n_out:n_in, ti:]
                rho_minus[n_out:n_in, :] = U.T.conj() @ rho_minus[n_out:n_in, :]
                rho_minus[:, n_out:n_in] = rho_minus[:, n_out:n_in] @ U

                U_cd.append(U.T.conj())
                n_out += 1

        if n_out < rho_minus.shape[0]:
            psi = la.as_column_vector(spread_cd[n_out:, ti])
            rho_minus[n_out:, n_out:] -= la.dyad(psi, psi) * dt
            la.make_hermitean(rho_minus)

    U_cd.append(None)
    return spread_cd, U_cd, times_out
    
def get_inout_range_dynamic(times_in, ti, times_out):
    m_in = lc.m_in(times_in, ti)
    m_out = sum(t_out <= ti for t_out in times_out)
    return m_out, m_in

from scipy.linalg import expm
import numpy as np

def moving_frame_dynamic(spread_cd, times_out, U_cd, dt):
    H_mv = []
    spread_mv = np.copy(spread_cd)

    for i, U in enumerate(U_cd):
        if U is None:
            H_mv.append(None)
            continue

        a = times_out[i-1]
        b = times_out[i]
        duration = (b - a) * dt

        e_, v_ = np.linalg.eig(U)
        e_ = np.log(e_ + 0j) / duration
        H = v_ @ np.diag(e_) @ v_.conj().T
        H_mv.append(H)

        n_out = i
        n_in = n_out + U.shape[0]

        U_step = np.eye(U.shape[0], dtype=complex)
        dU = expm(dt * H)

        for j in range(a, b):
            spread_mv[n_out:n_in, j] = U_step @ spread_mv[n_out:n_in, j]
            U_step = dU @ U_step

    return spread_mv, H_mv


def make_number_jump(psi, space, mode_index, reset_to_vac = True):
    # find the "preferred" jump basis
    rho = density_matrix(psi, space, mode_index)
    jump_probs = np.diag(rho)
    
    # select the jump 
    xi = random.random()
    jump_index = 0
    local_dim = len(jump_probs)
    for k in range(0, local_dim):
        xi = xi - jump_probs[k]
        if xi < 0:
            jump_index = k
            break
    
    psi_collapsed = np.zeros(space.dimension, dtype = complex)
    # apply the jump
    if reset_to_vac:
        psi_collapsed = psi_collapsed + space.outer(0, jump_index, mode_index) @ psi
    else:
        psi_collapsed = psi_collapsed + space.outer(jump_index, jump_index, mode_index) @ psi
        
    # normalize    
    psi_collapsed = psi_collapsed / np.sqrt(np.vdot(psi_collapsed, psi_collapsed))
    
    return psi_collapsed, jump_indeх

In [ ]:
n_sites = 100
es = [1]*n_sites
hs = [0.05]*(n_sites-1)
dt = 0.01
nt = 30000
spread = lc.spread(es, hs, nt, dt)
rho_plus = lc.rho_plus(spread, dt)

max_num_quanta = 5
n_samples = 2  

rtol_list = [1e-2, 1e-3, 1e-4, 1e-5, 1e-7, 1e-9]
FD_rtol = {}
hist_rtol = {}
for rtol in rtol_list:
    print(f"Run for rtol = {rtol}")
    # minimal forward light cone
    ti_arrival, spread_min, U_min, rho_plus_min = lc.minimal_forward_frame(spread, rho_plus, dt, rtol)
        
    # causal diamond frame
    spread_cd, U_cd, times_out = causal_diamond_frame_dynamic(spread_min, ti_arrival, U_min, rho_plus_min, dt, rtol)
        
    # moving frame
    spread_mv, H_mv = moving_frame_dynamic(spread_cd, times_out, U_cd, dt)
    # Time step
    dt = 0.01
    
    # Final grid
    t_max = 250
    t = np.arange(0, t_max + dt, dt)
    n_time = t.size
    
    num_modes = lc.m_in(ti_arrival, n_time - 1)
    m = models.spin_boson(num_modes, max_num_quanta)
    
    wq = 1.0
    Hs = wq * m.s_p @ m.s_m
    g = 0.05
    V = g * m.s_m
    V_dag = g * m.s_p
    
    def f(ti):
        return(0.1*np.cos((ti + 0.5)*dt))

    psi_0 = np.zeros(m.dimension, dtype = complex)
    psi_0[0] = 1
    all_psi_trajectories = []
    hist = []
    for sample in range(n_samples):
        hist_a = []
        np.random.seed(42 + sample)
        
        psi_t = np.zeros((n_time, m.dimension), dtype=complex)
        Ht = None
        Hint = m.space.zero_op
        m_out_prev = 0
            
        def begin_step(ti, psi):
            m_out, m_in = get_inout_range_dynamic(ti_arrival, ti, times_out)
            
            # if the number of decoupled modes is changed: we need to make a jump
            global m_out_prev
            if m_out != m_out_prev:
               psi[:], jump_index = make_number_jump(psi, m.space, m_out_prev + 1, reset_to_vac = False) # + 1 because the first mode is the qubit
               hist_a.append(jump_index)
               m_out_prev = m_out 
                
            global Hint
            if m_in > 0:
                Hint = V_dag @ sum(spread_mv[:num_modes, ti] * m.a[:])
                Hint = Hint + Hint.conj().transpose()
            global Ht
            Ht = Hs + m.s_x * f(ti) + Hint
            Hw = m.zero_op
            W = lc.get_H(ti_arrival, H_mv, ti)
            if W is not None:
                n_active = min(W.shape[0], m_in - m_out)
                a_loc = m.a[m_out:m_out + n_active]
                a_dag_loc = m.a_dag[m_out:m_out + n_active]
                for p in range(n_active):
                    for q in range(n_active):
                        Hw = 1j * a_dag_loc[q] @ a_loc[p] * W[q, p].conj()
            Ht = Ht + Hw
        
        def apply_h(ti, psi_in, psi_out):
            mv(Ht, psi_in, psi_out)
        
        def eval_o(ti, psi):
            psi_t[ti] = psi.copy()

        solve(0, n_time - 1, dt, apply_h, psi_0.copy(), begin_step=begin_step, eval_o=eval_o)
        all_psi_trajectories.append(psi_t)
        hist.append(hist_a)
        
    FD = np.zeros(n_time)
    for ti in range(n_time):
        FD[ti] = np.vdot(all_psi_trajectories[0][ti], all_psi_trajectories[1][ti]).real

    FD_rtol[rtol] = FD
    hist_rtol[rtol] = hist

In [ ]:
for rtol, FD in FD_rtol.items():
    plt.semilogy(t, FD, label=f"rtol = {rtol}")
plt.legend()